In [1]:
import * from pyspark

SyntaxError: invalid syntax (<ipython-input-1-4a81d6d303a9>, line 1)

In [2]:
from pyspark import *
sc =SparkContext.getOrCreate()

In [3]:
spark = SparkSession.builder.getOrCreate()

NameError: name 'SparkSession' is not defined

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.getOrCreate()

In [6]:
df = spark.read.json("dataset/people.json")

In [7]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [8]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [9]:
df.select(df.name,df.age+1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



In [10]:
df.filter(df.age>20).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [11]:
df.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



In [12]:
df.sort(df.age.desc()).show()

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+



In [13]:
df.select(df.name.alias("username"),df.age).show

<bound method DataFrame.show of DataFrame[username: string, age: bigint]>

In [14]:
df.select(df.name.alias("username"),df.age).show()

+--------+----+
|username| age|
+--------+----+
| Michael|null|
|    Andy|  30|
|  Justin|  19|
+--------+----+



In [15]:
from pyspark.sql.types import Row

In [16]:
def f(x):
    rel={}
    rel['name']=x[0]
    rel['age']=x[1]
    return rel

In [17]:
peopleDF=sc.textFile("dataset/people.txt")

In [18]:
peopleDF=peopleDF.map(lambda line:line.split(',')).map(lambda x: Row(**f(x))).toDF()

In [19]:
peopleDF.collect()

[Row(age=' 29', name='Michael'),
 Row(age=' 30', name='Andy'),
 Row(age=' 19', name='Justin')]

In [20]:
peopleDF.createOrReplaceGlobalTempView("people")

In [21]:
personsDF=spark.sql("select * from people")

AnalysisException: 'Table or view not found: people; line 1 pos 14'

In [22]:
peopleDF.createOrReplaceTempView("people")

In [23]:
personsDF = spark.sql("select * from people")

In [24]:
personsDF.collect()

[Row(age=' 29', name='Michael'),
 Row(age=' 30', name='Andy'),
 Row(age=' 19', name='Justin')]

In [25]:
personsDF.rdd.map(lambda t: "name"+t[0]+","+"Age:"+t[1].foreach(print))

PythonRDD[44] at RDD at PythonRDD.scala:53

In [26]:
personsDF.rdd.map(lambda t: "name"+t[0]+","+"Age:"+t[1]).collect()

['name 29,Age:Michael', 'name 30,Age:Andy', 'name 19,Age:Justin']

In [27]:
from pyspark.sql.types import StructType

In [28]:
from pyspark.sql.types import StructField

In [29]:
from pyspark.sql.types import StringType

In [30]:
peopleRDD=sc.textFile("dataset/people.txt")

In [31]:
schemaString = "name age"

In [32]:
fields = list(map(lambda fieldName : StructField(fieldName,StringType(),nullable=true),schemaString.split(" ")))

NameError: name 'true' is not defined

In [33]:
fields = list(map(lambda fieldName : StructField(fieldName,StringType(),nullable=True),schemaString.split(" ")))

In [34]:
schema = StructType(fields)

In [35]:
rowRDD = peopleRDD.map(lambda line : line.split(',').map(lambda attributes : Row(attributes[0],attributes[1])))

In [36]:
peopleDF = spark.createDataFrame(rowRDD,schema)

In [37]:
peopleDF.createOrReplaceTempView("people")

In [38]:
results = spark.sql("select * from people")

In [39]:
results.rdd.map( lambda attributes : "name: " + attributes[0]+","+"age:"+attributes[1]).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 20.0 failed 1 times, most recent failure: Lost task 1.0 in stage 20.0 (TID 215, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-35-0e9ca7d54466>", line 1, in <lambda>
AttributeError: 'list' object has no attrib

In [40]:
results.rdd.map(lambda t: "name"+t[0]+","+"Age:"+t[1]).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 21.0 failed 1 times, most recent failure: Lost task 1.0 in stage 21.0 (TID 217, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-35-0e9ca7d54466>", line 1, in <lambda>
AttributeError: 'list' object has no attrib

In [41]:
results.collect()

Py4JJavaError: An error occurred while calling o182.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 22.0 failed 1 times, most recent failure: Lost task 0.0 in stage 22.0 (TID 218, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-35-0e9ca7d54466>", line 1, in <lambda>
AttributeError: 'list' object has no attribute 'map'

	at org.apache.spark.api

In [42]:
peopleDF.collect()

Py4JJavaError: An error occurred while calling o181.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 23.0 failed 1 times, most recent failure: Lost task 1.0 in stage 23.0 (TID 221, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-35-0e9ca7d54466>", line 1, in <lambda>
AttributeError: 'list' object has no attribute 'map'

	at org.apache.spark.api

In [43]:
peopleRDD.collect()

['Michael, 29', 'Andy, 30', 'Justin, 19']

In [44]:
schemaString = "name age"

In [45]:
fields = list(map( lambda fieldName : StructField(fieldName, StringType(), nullable = True), schemaString.split(" ")))

In [46]:
schema = StructType(fields)

In [47]:
rowRDD = peopleRDD.map(lambda line : line.split(',')).map(lambda attributes : Row(attributes[0], attributes[1]))

In [48]:
peopleDF = spark.createDataFrame(rowRDD, schema)

In [49]:
peopleDF.createOrReplaceTempView("people")

In [50]:
results = spark.sql("SELECT * FROM people")

In [51]:
results.rdd.map( lambda attributes : "name: " + attributes[0]+","+"age:"+attributes[1]).foreach(print)

In [52]:
results.rdd.map( lambda attributes : "name: " + attributes[0]+","+"age:"+attributes[1]).collect()

['name: Michael,age: 29', 'name: Andy,age: 30', 'name: Justin,age: 19']

In [53]:
peopleDF = spark.read.format('json').load("dataset/people.json")

In [54]:
peopleDF.select("name","age").write.format("dataset/people.csv")

In [55]:
peopleDF.select("name","age").write.format("csv").save("dataset/newpeople.csv")

In [56]:
peopleDF.rdd.saveAsTextFile("dataset/newpeople.txt")